In [ ]:
# --- Step 1: Install Tools ---
# We need to download some special tools (libraries) to help us work.
# 'pip install' is like going to an app store and downloading an app.

# 1. 'pandas': A super powerful tool for working with data tables (like Excel).
!pip install -q pandas==2.2.2

# 2. 'google-generativeai': The brain! This connects us to Google's AI models.
!pip install -q -U google-generativeai

# 3. 'matplotlib' and 'seaborn': These are our artists. They help draw charts and graphs.
!pip install -q -U matplotlib
!pip install -q -U seaborn

# 4. 'tabulate': Helps make our data look neat and tidy in tables.
!pip install -q -U tabulate

# 5. 'langchain': This is like a manager that helps different tools talk to each other.
!pip install -q -U langchain
!pip install -q -U langchain-community
!pip install -q -U langchain-experimental
!pip install -q -U "langchain-core>=1.0.0"
!pip install -q -U langchain-google-genai

# 6. 'tenacity' and 'packaging': Helper tools to make sure things run smoothly and don't crash easily.
!pip install -q -U tenacity
!pip install -q -U packaging

In [ ]:
# --- Step 2: Bring the Tools to the Workbench ---
# Now that we downloaded the tools, we need to put them on our table to use them.
# 'import' tells the computer "Hey, get ready to use this tool!"

# Import pandas for data handling (we'll call it 'pd' for short)
import pandas as pd

# Import plotting tools (our artists)
import matplotlib.pyplot as plt
import seaborn as sns

# Import a tool to interact with the computer's system
import os

# Import a tool to handle warnings (like 'be quiet' mode)
import warnings
# Tell the system to ignore annoying orange warning boxes that aren't errors
warnings.filterwarnings('ignore')

# Import tools to talk to Google Colab and Google's AI
from google.colab import userdata
import google.generativeai as genai

# Import the specific AI agent that knows how to read dataframes (tables)
from langchain_experimental.agents.agent_toolkits import create_pandas_dataframe_agent

# Import the chat interface for Google's AI
from langchain_google_genai import ChatGoogleGenerativeAI

# Set a nice visual style for our charts (optional, but looks better!)
sns.set_theme(style="whitegrid")

In [ ]:
# --- Step 3: Unlock the AI ---
# We need a secret password (API Key) to use Google's powerful AI.

try:
    # 1. Try to get the secret key from your Colab 'Secrets' vault.
    GOOGLE_API_KEY = userdata.get('GOOGLE_API_KEY')

    # 2. Configure the AI with this key so it knows it's us.
    genai.configure(api_key=GOOGLE_API_KEY)

    # 3. Also tell the system about this key so other tools can find it.
    os.environ["GOOGLE_API_KEY"] = GOOGLE_API_KEY

    # 4. If everything works, tell the user!
    print("API Key successfully loaded.")

except Exception as e:
    # 5. If something goes wrong (like you forgot to set the key), show this error message.
    print("Error loading API Key. Please make sure 'GOOGLE_API_KEY' is set in Colab Secrets.")

In [ ]:
# --- Step 4: Get Your Data ---
# We need to load the Excel/CSV file you want to talk to.

from google.colab import files

# 1. Ask the user to verify and upload a file
print("Please upload your CSV file:")
uploaded = files.upload()

# 2. Get the name of the file you just uploaded
filename = next(iter(uploaded))

try:
    # 3. Read the file into a 'DataFrame' (think of it as a smart spreadsheet in memory)
    df = pd.read_csv(filename)

    # 4. Confirm it worked
    print(f"\nSuccessfully loaded '{filename}'")

    # 5. Show the first few rows so you can see what the data looks like
    print("Data Preview:")
    display(df.head())

except Exception as e:
    # 6. If the file is bad/corrupt, tell us what went wrong
    print(f"Error loading CSV: {e}")

In [ ]:
# --- Step 5: Check Available Brains (Optional) ---
# See which AI models are available to us correctly.

# Loop through all models Google offers
for m in genai.list_models():
  # We only want models that can 'generateContent' (write text/answers)
  if "generateContent" in m.supported_generation_methods:
    # Print the name of the model
    print(m.name)

In [ ]:
# --- Step 6: Create the AI Assistant ---
# Now we build the "Agent" that connects the AI brain to your data.

# 1. Initialize the LLM (Large Language Model) - the brain.
#    We use 'gemini-flash-lite-latest' because it's fast and smart.
#    'temperature=0' means "be precise and factual, don't be creative".
llm = ChatGoogleGenerativeAI(
    model="gemini-flash-lite-latest",
    temperature=0,
    max_output_tokens=2048
)

# 2. Create the Agent.
#    This connects the LLM (brain) to the 'df' (your data).
#    'allow_dangerous_code=True' lets the AI write Python code to calculate answers (necessary for math!)
agent = create_pandas_dataframe_agent(
    llm,
    df,
    verbose=False,
    allow_dangerous_code=True
)

print("Agent initialized successfully! You can now ask questions.")

In [ ]:
# --- Step 7: Define HOW to Ask Questions ---
# We create a simple function (a reusable command) to handle asking and showing answers.

def ask_agent(query):
    # 1. Tell the user we are thinking...
    print(f"Processing Query: '{query}'...\n")

    try:
        # 2. Send the question (query) to our Agent and get the response
        response = agent.run(query)

        # 3. Print the text answer
        print("\nAnswer:")
        print(response)

        # 4. Check if the AI drew a chart. If yes, show it!
        if plt.get_fignums():
            plt.show()
        else:
            # If no chart, do nothing extra
            pass

    except Exception as e:
        # 5. If the AI gets confused or errors, tell us what happened
        print(f"An error occurred: {e}")

# Test it with a simple question immediately to make sure it works
query = "Show me a distribution of the numerical columns"
ask_agent(query)

In [ ]:
# --- Step 8: Start the Chat Loop ---
# This keeps the program running so you can ask question after question.

while True:
    # 1. Wait for you to type a question
    user_input = input("\nAsk a question (or type 'exit' to quit): ")

    # 2. Check if you want to stop the program
    if user_input.lower() in ['exit', 'quit']:
        break

    # 3. If not stopping, send your question to the agent helper we made above
    ask_agent(user_input)